In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [1]:
import numpy as np

import ccm_coda, kernel
import kernel
import tensorflow as tf

import pickle, os

from tools import * # we use cv_score, zero_replacement functions here

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2023-05-23 00:53:08.280584: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 00:53:08.417272: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-23 00:53:09.980308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/python/3.7.2-tf/lib:/opt/gcc/5.4.0/lib64:/home/apps/cuda-10.2/lib64:/home/apps/cud

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12500391619598031885
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23367450624
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9175531477664728787
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:18:00.0, compute capability: 8.6"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 23367450624
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13899808925466518644
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:51:00.0, compute capability: 8.6"
xla_global_id: 2144165316
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 23367450624
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15749649558754070239
physical_device_desc: "device: 2, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:8a:00.0, compute capability: 8.6"
xla_global_id: 1651660799
]


2023-05-23 00:53:12.300956: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 00:53:15.608600: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:0 with 22284 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:18:00.0, compute capability: 8.6
2023-05-23 00:53:15.609028: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:1 with 22284 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:51:00.0, compute capability: 8.6
2023-05-23 00:53:15.609301: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:2 with 22284 MB memory:  -> device: 2, name: NVIDIA GeForce RT

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import composition_stats as cp

from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, StratifiedKFold, train_test_split, LeaveOneOut
from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, get_scorer, r2_score, mean_squared_error

from scipy.spatial.distance import pdist

import warnings
warnings.filterwarnings("ignore")

## COMBO dataset
Dataset downloaded from https://c-lasso.readthedocs.io/en/latest/auto_examples/plot_example_COMBO.html  

In [3]:
count = pd.read_csv('./datasets/COMBO_data/complete_data/GeneraCounts.csv', header=None)
count = count.transpose()

tax_table = pd.read_csv('./datasets/COMBO_data/complete_data/GeneraPhylo.csv', header=None)
tax_table = tax_table.transpose()
tax_table.columns = tax_table.iloc[0]
tax_table.drop(tax_table.index[0], inplace=True)
tax_table

,OTU_1,OTU_2,OTU_3,OTU_4,OTU_5,OTU_6,OTU_7,OTU_8,OTU_9,OTU_10,...,OTU_78,OTU_79,OTU_80,OTU_81,OTU_82,OTU_83,OTU_84,OTU_85,OTU_86,OTU_87
1,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,...,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria
2,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Bacteroidetes,Bacteroidetes,Bacteroidetes,...,Proteobacteria,Proteobacteria,Proteobacteria,Proteobacteria,Proteobacteria,Proteobacteria,Proteobacteria,Synergistetes,Synergistetes,Verrucomicrobia
3,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Bacteroidia,Bacteroidia,Bacteroidia,...,Betaproteobacteria,Betaproteobacteria,Deltaproteobacteria,Epsilonproteobacteria,Gammaproteobacteria,Gammaproteobacteria,Gammaproteobacteria,Synergistia,Synergistia,Verrucomicrobiae
4,Coriobacteriales,Coriobacteriales,Coriobacteriales,Coriobacteriales,Coriobacteriales,Coriobacteriales,Coriobacteriales,Bacteroidales,Bacteroidales,Bacteroidales,...,Burkholderiales,Neisseriales,Desulfovibrionales,Campylobacterales,Aeromonadales,Pseudomonadales,Xanthomonadales,Synergistales,Synergistales,Verrucomicrobiales
5,Coriobacteriaceae,Coriobacteriaceae,Coriobacteriaceae,Coriobacteriaceae,Coriobacteriaceae,Coriobacteriaceae,Coriobacteriaceae,Bacteroidaceae,Porphyromonadaceae,Porphyromonadaceae,...,Oxalobacteraceae,Neisseriaceae,Desulfovibrionaceae,Campylobacteraceae,Succinivibrionaceae,Pseudomonadaceae,Xanthomonadaceae,Synergistaceae,Synergistaceae,Verrucomicrobiaceae
6,Asaccharobacter,Atopobium,Collinsella,Eggerthella,Gordonibacter,Olsenella,Slackia,Bacteroides,Barnesiella,Butyricimonas,...,Oxalobacter,Neisseria,Desulfovibrio,Campylobacter,Succinivibrio,Pseudomonas,Stenotrophomonas,Cloacibacillus,Pyramidobacter,Akkermansia


In [4]:
# BMI as response
Y = pd.read_csv('./datasets/COMBO_data/BMI.csv', header=None)
Y = np.array(Y).ravel()

# Zero ratio, make composotional
X = np.array(count, dtype=float)
print("zero ratio:", np.sum(X == 0) / X.size)
X = X / np.sum(X, axis=1)[:, None]

zero ratio: 0.7209051724137931


To make comparison with GBM zero replacement, we delete columns with only one nonzero value

In [5]:
valid_idx = np.sum(X > 0, axis=0) > 1

X = np.array(count.iloc[:, valid_idx], dtype=float)
print(X.shape)
print("zero ratio:", np.sum(X == 0) / X.size)
X = X / np.sum(X, axis=1)[:, None]

(96, 77)
zero ratio: 0.6860119047619048


## CV-MSE computation in conjunction with Kernel Ridge Regression
Rewrite this section using the Section "Using Column-deleted Data"

In [9]:
for m in [3, 5, 10, 77]:
    cv_score(X, Y - Y.mean(0), "real-valued", num_features=m, model_name="krr", savepath="bmi_final",
             epsilon = 0.1, learning_rate = 0.001, 
             iterations = 5000, verbose=True,
             outer_folds = 5, inner_folds = 5, N_TRIALS=10,
             do_alpha=True, no_gamma=False, radial=True, oneSE=True)


 1 th cv experiment...

 Fold 0:


2023-05-23 00:03:28.581812: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22284 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:18:00.0, compute capability: 8.6
2023-05-23 00:03:28.582592: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22284 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:51:00.0, compute capability: 8.6
2023-05-23 00:03:28.582748: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 22284 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:8a:00.0, compute capability: 8.6
2023-05-23 00:03:32.357906: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-23 00:03:32.383086: 


iteration 100 loss 229.63062
iteration 200 loss 227.80814
iteration 300 loss 227.47948
iteration 400 loss 227.3699
iteration 500 loss 227.32782
iteration 600 loss 227.30269
iteration 700 loss 227.28696
iteration 800 loss 227.27676
iteration 900 loss 227.2699
iteration 1000 loss 227.2653
iteration 1100 loss 227.2621
iteration 1200 loss 227.25987
iteration 1300 loss 227.2583
iteration 1400 loss 227.25723
iteration 1500 loss 227.25644
iteration 1600 loss 227.25589
iteration 1700 loss 227.25548
iteration 1800 loss 227.25522
iteration 1900 loss 227.25504
iteration 2000 loss 227.25484
iteration 2100 loss 227.25476
iteration 2200 loss 227.2547
iteration 2300 loss 227.25467
iteration 2400 loss 227.25462
iteration 2500 loss 227.2546
iteration 2600 loss 227.25458
iteration 2700 loss 227.25458
iteration 2800 loss 227.25456
iteration 2900 loss 227.25455
iteration 3000 loss 227.25455
iteration 3100 loss 227.25455
iteration 3200 loss 227.25453
iteration 3300 loss 227.25455
iteration 3400 loss 227.2

In [14]:
selections = dict()
for m in [3, 5, 10, 77]:
    with open("./results/real_data/bmi_final/krr_neg_mean_squared_error_{}_1seTrue_radial.pickle".format(m), "rb") as fi:
        score, selection = pickle.load(fi)
    score_cv = np.mean(score, axis=1)
    print(np.mean(score_cv), np.std(score_cv)/np.sqrt(10))
    if m == 77:
        break
    selections[m] = selection_wrapper(selection, 77)
    # m = 77 (dim of X) is for comparison

28.90191912108356 0.053039353626440376
Selected features are: [15 40 48]
28.873559945159844 0.037140273387692165
Selected features are: [14 15 40 48 58]
28.98394520611011 0.11514238873274053
Selected features are: [ 9 14 15 40 48 54 56 58 63 69]
29.29507497131206 0.29381417600100346


In [15]:
for m in [3, 5, 10]:
    print("{} variables\n\n".format(m), tax_table.iloc[:, selections[m]['Selected_features']], "\n")

3 variables

 0          OTU_16            OTU_41            OTU_49
1        Bacteria          Bacteria          Bacteria
2   Bacteroidetes        Firmicutes        Firmicutes
3     Bacteroidia        Clostridia        Clostridia
4   Bacteroidales     Clostridiales     Clostridiales
5   Rikenellaceae   Lachnospiraceae   Ruminococcaceae
6       Alistipes             Dorea     Anaerotruncus 

5 variables

 0           OTU_15          OTU_16            OTU_41            OTU_49  \
1         Bacteria        Bacteria          Bacteria          Bacteria   
2    Bacteroidetes   Bacteroidetes        Firmicutes        Firmicutes   
3      Bacteroidia     Bacteroidia        Clostridia        Clostridia   
4    Bacteroidales   Bacteroidales     Clostridiales     Clostridiales   
5   Prevotellaceae   Rikenellaceae   Lachnospiraceae   Ruminococcaceae   
6       Prevotella       Alistipes             Dorea     Anaerotruncus   

0            OTU_59  
1          Bacteria  
2        Firmicutes  
3      

In [ ]:
new_score = list()
for m in [3, 5, 10, 77]:
    with open("./results/real_data/bmi_rm/krr_neg_mean_squared_error_{}_1seTrue_radial.pickle".format(m), "rb") as fi:
        score, selection = pickle.load(fi)

    score = cv_with_selection(X, Y - Y.mean(0), 'real-valued', selection, savepath="bmi3",
                              proj_method = 'compo', model_name='krr',
                              inner_folds=5, do_alpha=True, no_gamma=False, 
                              radial=True, oneSE=True)
    new_score.append(score)
new_score

In [ ]:
for m in [3, 5, 10, 77]:
    with open("./results/real_data/bmi3/krr_neg_mean_squared_error_{}_1seTrue_radial.pickle".format(m), "rb") as fi:
        score, selection = pickle.load(fi)
    score_cv = np.mean(score, axis=1)
    print(np.mean(score_cv), np.std(score_cv)/np.sqrt(10))

28.90191912108356 0.053039353626440376 (10,)
28.873559945159844 0.037140273387692165 (10,)
28.98394520611011 0.11514238873274053 (10,)
29.29507497131206 0.29381417600100346 (10,)


## Coda-lasso experiments with the c-lasso module

In [17]:
from classo import classo_problem
  
def learn(Z, Y, C=None, label=None, lam="theoretical"):
    # Z: log-transformed compositional data
    # Default C: zero sum constraint
    problem = classo_problem(Z, Y, C, label=label)
    problem.formulation.concomitant = False
    problem.formulation.huber = False
    problem.formulation.intercept = True

    problem.model_selection.CV = False
    problem.model_selection.ALO = False
    problem.model_selection.StabSel = False

    problem.model_selection.LAMfixed = True
    problem.model_selection.LAMfixedparameters.lam = lam
    problem.solve()
    
    return problem


def predict(model, X_new):
    beta = model.solution.LAMfixed.beta
    X_new = np.c_[np.ones(len(X_new)), X_new]
    return X_new @ beta

In [18]:
def cv_lasso_regression(X, Y, savepath='bmi_final', C=None, label=None, lamseq=np.geomspace(1e-3, 1, 100),
                   outer_folds = 5, inner_folds = 5, oneSE=False, N_TRIALS = 10, **krgs,
                   ):
    '''
    Input X should be zero-replaced count or compositional data
    label=None is almost always better.. it returns indices
    '''
    assert (X > 1e-10).all(), "Replace zeros or check the data type"
    X = np.log(X)
    
    if label is None:
        label = np.arange(X.shape[1])

    score_all = list()
    selection_all = list()
    
    for seed_ in range(1, N_TRIALS + 1):
        print("\n", seed_, "th cv experiment...")

        # Outer cv folds
        Outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=seed_)
        score = list()
        selection = list()
        
        # Outer cv loop
        for i, (train_idx, test_idx) in enumerate(Outer_cv.split(X)):
            print(f"\n Fold {i}:")
            X_train, Y_train = X[train_idx], Y[train_idx]
            X_test, Y_test = X[test_idx], Y[test_idx]

            if inner_folds is None:
                raise Exception("Not yet developed")
            else:
                # Do inner cv on X_train to choose lambda parameter
                inner_cv = KFold(n_splits=inner_folds, shuffle=True, random_state=seed_ * N_TRIALS + i)
                cv_scores = np.zeros((len(lamseq), inner_folds))
                
                for j, (tr, val_idx) in enumerate(inner_cv.split(X_train)):
                    for k in range(len(lamseq)):
                        lam = lamseq[k]
                        model = learn(X_train[tr], Y_train[tr], lam=lam)
                        cv_scores[k, j] = mean_squared_error(Y_train[val_idx],
                                                            predict(model, X_train[val_idx]))
                
                # Parameter selection based on inner_cv result
                cv_scores_collected = cv_scores.mean(axis=1)
                
                if oneSE:
                    # Take maximal lambda among the [min, min+1se] scores
                    min_plus_1se = np.min(cv_scores_collected) + np.std(cv_scores_collected) / np.sqrt(len(cv_scores_collected))
                    mask = cv_scores_collected < min_plus_1se
                    lamb = lamseq[mask][-1]
                    # print("models:", lamseq[mask])
                else:
                    # find lambda "minimizing" cv-mse
                    lamb = lamseq[np.argmin(cv_scores_collected)]
                # Refit
                refitted_model = learn(X_train, Y_train, label=label, lam=lamb)
                
            # Get score
            refitted_score = mean_squared_error(Y_test,
                                                predict(refitted_model, X_test))
            selected_feats = label[refitted_model.solution.LAMfixed.selected_param[1:]]
            print("CV lambda, refitted score:", lamb, ',', refitted_score)
            print("Selected feats:", selected_feats)
            score.append(refitted_score)
            selection.append(selected_feats)
        
        # Wrap up the result of _ th iteration
        score_all.append(score)
        selection_all.append(selection)
    
    print(np.mean(score_all), "\n")
    
    # Save
    with open("./results/real_data/{}/cvlasso_inner{}_1se{}.pickle".format(savepath, inner_folds, oneSE), 'wb') as f:
        pickle.dump((score_all, selection_all), f)
    return score_all
        

In [19]:
# 0.5min replacement
X_replaced = zero_replacement(X, method='min', val=1/2)
cv_lasso_regression(X_replaced, Y, savepath='bmi_final', label=None, inner_folds=5)


 1 th cv experiment...

 Fold 0:
CV lambda, refitted score: 0.49770235643321137 , 40.55090725999977
Selected feats: [15 52]

 Fold 1:
CV lambda, refitted score: 0.21544346900318845 , 7.730618565570331
Selected feats: [24 52 53]

 Fold 2:
CV lambda, refitted score: 0.26560877829466867 , 38.15574175852951
Selected feats: [15 52]

 Fold 3:
CV lambda, refitted score: 0.23101297000831603 , 41.76653427968882
Selected feats: [24 52 53]

 Fold 4:
CV lambda, refitted score: 0.23101297000831603 , 15.401680541540708
Selected feats: [24 48 52 62]

 2 th cv experiment...

 Fold 0:
CV lambda, refitted score: 0.8697490026177834 , 40.08073394600363
Selected feats: []

 Fold 1:
CV lambda, refitted score: 0.3511191734215131 , 21.214318155627268
Selected feats: [15 52]

 Fold 2:
CV lambda, refitted score: 0.37649358067924676 , 30.63558627459497
Selected feats: [24 52]

 Fold 3:
CV lambda, refitted score: 0.2009233002565048 , 26.0703556539422
Selected feats: [15 24 37 52 53]

 Fold 4:
CV lambda, refitted

[[40.55090725999977,
  7.730618565570331,
  38.15574175852951,
  41.76653427968882,
  15.401680541540708],
 [40.08073394600363,
  21.214318155627268,
  30.63558627459497,
  26.0703556539422,
  32.146656298324494],
 [28.282221749587563,
  40.728216196633404,
  14.230316445254008,
  17.850762211088437,
  52.77894810313268],
 [49.79001823285363,
  15.07130757217668,
  28.653685626842048,
  20.175023945668777,
  33.205098292361704],
 [26.090993274349234,
  21.814217952070738,
  51.68432325426316,
  26.72120509735529,
  24.030108899491378],
 [45.25825292448061,
  31.142228009845894,
  30.21089605888627,
  9.395862458454548,
  24.477913909647977],
 [20.54011881103027,
  27.077764301767797,
  33.191871914673676,
  29.3684164237305,
  38.57122906390675],
 [30.3986186390984,
  24.75909938042152,
  43.30045162652417,
  27.23751183540398,
  18.05856411674996],
 [21.012251813613403,
  50.240664311508965,
  33.237956779375025,
  14.417909033007579,
  18.895486377292038],
 [31.73099124446215,
  21.1

In [24]:
with open("./results/real_data/{}/cvlasso_inner{}_1se{}_0.5min.pickle".format("bmi_final", 5, False), 'rb') as fi:
    score, selection = pickle.load(fi)
mse_through_repeat = np.mean(score, axis=1)
mse_through_repeat.mean(), mse_through_repeat.std() / np.sqrt(10)

(29.28863623980923, 0.2969653109467116)

As shown below, applying 1se rule for CV returns almost no selection but intercept (lasso isn't prone to overfitting)  
We only record CV result without applying 1se rule for lasso

In [ ]:
# 1se applied
X_replaced = zero_replacement(X, method='min', val=1/2)
cv_lasso_regression(X_replaced, Y, savepath='bmi_final', label=None, inner_folds=5, oneSE=True)

In [25]:
with open("./results/real_data/{}/cvlasso_inner{}_1se{}_0.5min.pickle".format("bmi_final", 5, True), 'rb') as fi:
    score, selection = pickle.load(fi)
mse_through_repeat = np.mean(score, axis=1)
mse_through_repeat.mean(), mse_through_repeat.std() / np.sqrt(10)

(29.58014152071525, 0.16147063989710317)

In [26]:
# lasso with 1se rule asserts almost no selection is the best
selection

[[array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([24, 48, 52]),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([24, 56]),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dty

### 1sum replacement

In [ ]:
# 1sum replacement
X_replaced = zero_replacement(X, method='sum', val=1.)
cv_lasso_regression(X_replaced, Y, savepath='bmi_final', label=None, inner_folds=5)

In [29]:
with open("./results/real_data/{}/cvlasso_inner{}_1se{}_1sum.pickle".format("bmi_final", 5, False), 'rb') as fi:
    score, selection = pickle.load(fi)
mse_through_repeat = np.mean(score, axis=1)
mse_through_repeat.mean(), mse_through_repeat.std() / np.sqrt(10)

(30.516218621834316, 0.37851058876287474)

### GBM replacement

In [ ]:
# Used R package zCompositions
x_gbm = pd.read_csv('./datasets/COMBO_data/complete_data/GeneraCountsGBM.csv')
x_gbm = np.array(x_gbm.iloc[:, 1:])
cv_lasso_regression(x_gbm, Y, savepath='bmi_final', label=None, inner_folds=5)

In [32]:
# gbm result
with open("./results/real_data/{}/cvlasso_inner{}_1se{}_gbm.pickle".format("bmi_final", 5, False), 'rb') as fi:
    score, selection = pickle.load(fi)
mse_through_repeat = np.mean(score, axis=1)
mse_through_repeat.mean(), mse_through_repeat.std() # has to / np.sqrt(10)

(29.052951227592416, 1.391898494377944)